ANDROİD GAMES isimli VERİ TABANI üzerinde çalışma yapılmıştır. 
    - Öncelikle veri tablosundaki alanların veri tipleri ve içindeki veri bütünlüğü ele alınmıştır.
    - Tablolar arasındaki ilişkiler ve etkileri ele alınmıştır.
    - Veriler görseleştirilmiş ve yorumlanabilir hale getirilmiştir.
    
    
 Android games offline data on the base were performed. 
 - First, the data types of fields in the data table and the data integrity in them are discussed.
 - Relations and effects between tables are decoded.
 - Data has been visualized and interpreted.

If your plotly module is not in your working environment, please download plotly and run the following codes.
First --> pip install plotly==5.1.0
Then --> import plotly.io as pio and pio.renderers.default = 'iframe'

In [ ]:
import numpy as np
import pandas as pd
# Görselleştirme
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
# sql
import pandas.io.sql
from datetime import datetime
now = datetime.now() # current date and time


In [ ]:
df = pd.read_csv('../input/top-play-store-games/android-games.csv')

In [ ]:
df.head(2)

In [ ]:
df.shape   # 15 farklı kategoride 1730 oyun olduğunu görüyoruz.

Verinin incelenmesi çalışnmalarına başlıyoruz. Öncelikle NULL verileri kontrol etmeliyiz. isnull komutuyla veri tabanı içinde NULL kayıt olmadığını görüyoruz. Bu durum verinin incelenmesi açısından oldukça iyi bir durum. Şu haliyle bir sürpriz ile karşılaşmazsak temiz veri olduğu düşünülebilir.

We're starting to study the data. First, we must check the NULL data. with the isnull command, we see that there are no NULL records in the database. This is a pretty good situation in terms of studying the data. As it stands, if we don't face a surprise, it can be considered clean data Juliette.

In [ ]:
df.isnull().sum()

Veri tabanının yapısını incelediğimizde incelenebilecek alanların "RANK", "TİTLE", "TOTAL RATING", "İNSTALLS", "CATEGORY", "PRICE" olduğu ilk bakışta görülmektedir. Anakiz ilerledikçe farı alternatifler düşünülebilir.

RANK - TOTAL RATING İLİŞKİSİ
CATEGORY - RANK İLİŞKİSİ
İNSTALLS - RATİNG İLİŞKİSİ
PİRCE - İNSTALL İLİŞKİSİ

öncelikli olarak analiz edilecek ve yorumlanacak hususlardır.

--------------------

When we examine the structure of the database, it is seen at first glance that the fields that can be examined are "RANK", "TITLE", "TOTAL RATING", "INSTALLS", "CATEGORY", "PRICE". As Anakiz progresses, the shadow can be considered alternatives.

RANK-TOTAL RATING RELATIONSHIP
CATEGORY-RANK RELATIONSHIP
INSTALLS-RATING RELATIONSHIP
PIRCE-INSTALL RELATIONSHIP

first of all, they are issues that will be analyzed and interpreted.

In [ ]:
df.info()

RANK alanı şu aşamada şu aşamada anlamsız gibi görünüyor, 
TOTAL RATİNG yorumlama açısından önemli alanlardan birisi, 
DESCRİBE ile sayısal veriler incelendiğinde MXİMUM OUTLİER olduğu görülmektedir.
MEAN ve MEDİAN arasındaki veri dikkate alınacak kadar büyük.
Mean değerini yükselten maksimum değer olmalı.Total Rating değeri çok yüksek olan oyunlar var. Dolayısıyla VARAGE RATİNG kesinlikle incelenmeli.

PRİCE verileri hayli ilginç, %99 luk bölümü 0 değerinden oluşuyor. Bu veriden oyunların büyük bölümünün ücretsiz olduğu anlaşılıyor. PRİCE alanını diğer alanlarla ilişkisi daha iyi incelenmeli.

---------------------

The RANK field seems to be meaningless at this stage, one of the important areas in terms of total RATING interpretation is mximum OUTLIER when examining numerical data with DESCRIBE. The data between MEAN and MEDIAN is large enough to be taken into account.Dec. It must be the maximum value that increases the Mean value.There are games with a very high Total Rating. Therefore, VARAGE RATING should definitely be examined.

PRICE data is very interesting, 99% of the part consists of 0. From this data, it seems that most of the games are free. Price's relationship to other fields should be better studied.

In [ ]:
df.describe() 

INSTALLS alanını incelediğimizde oyunların install bilgileri görülmektedir. Ancak verialanı OBJECT tipindedir. İçeriğinde ise rakamlarla beraber harflerde bulunmaktadır. Budurm Analizi etkileyeceğinde bu alan üzerinde aynılaştırma çalışması yapılması gerekmektedir. Bu alandaki veriler sayısal değere çevrilip anlamlı hale getirilmelidir.

Aynılaştırma çalışması yapıldıktan sonra CATEGORY alanıyla ilişkisi üzerinde durulmalıdır.

--------------


When we examine the INSTALLS field, we see the install information of the games. However, the data is of the OBJECT type. In its content, it is in letters with numbers. If this will affect the analysis, it is necessary to do the same work on this area. Data in this field should be translated to numeric value and made meaningful.

After doing the same study, the relationship with the CATEGORY field should be emphasized.

Veri içeriğinde BİN aralığını ifade eden K harfi, MİLYON aralığını ifade eden M harfinin olduğu görülmektedir. Öncelikle K harfinin değerleri M harfine dönüştürülmeli, Sonrasındada M harfi düşürüldüğünde sayısal veriler analiz edilebilir duruma gelecektir. 

-------------------

In December, the data is shown to contain the letter K, which refers to the December thousand, and the letter M, which refers to the range of million. First, the values of the letter K must be converted to the letter M, and then, when the letter M is reduced, the numerical data will be analyzed.

In [ ]:
df["installs"].value_counts()

In [ ]:
def in_thousand (inst):
    if inst == '500.0 k':
        return '0.5 M' 
    elif inst == '100.0 k':
        return '0.1 M'
    else:
        return inst
df['installs']= df['installs'].apply(in_thousand)

df['installs']= df['installs'].str.replace( 'M', '').str.strip().astype('float')

df= df.rename(columns={'installs': 'installs_in_million'})
df['installs_in_million'].value_counts()

PRİCE ve PAİD alanları üzerinde çalışalım. Bu alanlar incelendiğinde öncelikle veri tiplerinin farklı olduğu görülecektir. 
alanı incelendiğinde ürünlerin fiyatlarından sadece 7 tanesinde fiyat bilgisinin olduğu, diğerlerinin ücretsiz olması PRİCE ve PAİD alanındaki incelemeyi anlamsız hale getirmketedir. SAMPLE inceleme için kabul edilen minimum degeri 30 ile 50 arasında değiştiği düşünüldüğünde mevcut verinin yorumlama için yeterli olmayacağı aşikardır. Ancak diğer alanlarla ilişkidi değerlendirilebilir.  

-----------------------
Let's work on the price and PAID fields. When these fields are examined, it will be seen that the data types are different first. 
when examining the field, only 7 of the prices of products have Price Information, others are free of charge, making the review in the PRICE and PAID field meaningless. Given that the minimum accepted value for SAMPLE review varies between 30 and 50, it is obvious that the available data will not be sufficient for interpretation.Dec. But its relationship to other areas can be evaluated.

In [ ]:
df["price"].value_counts()

In [ ]:
df["price"].value_counts(normalize=True)

In [ ]:
df["paid"].value_counts()

PRICE alanı ve PAID alanı aynı olduğundan analiz için PAID alanı ile devam edileceğinden PRICE alanı DROP ile veri tabanından kaldırılmıştır.

---------

Since the price field and the PAID field are the same, the price Field has been removed from the database with DROP, as the analysis will continue with the PAID field.

In [ ]:
df.info()

ANALİZ BÖLÜMÜNE BAŞLAYALIM

CATEGORY alanı analiz ve yorumlamanın en önemli alanlarından birisi olacaktır. CATEGORY alanını merkeze koyup öncelikle diğer alanlarla ilişkisi yorumlanacaktır. Veri tabanında yaklaşık 100 oyun kategorisinin verileri bulunmaktadır. 

-------------------
The CATEGORY field will be one of the most important areas of analysis and interpretation. The CATEGORY field will be placed in the center and its relationship with other fields will be interpreted first. The database contains data from about 100 game categories.

In [ ]:
df["category"].value_counts()

Görselleştirme araçlarıyla çalışmaya başlayalım.

In [ ]:
plt.figure(figsize=(16,5)) 
sns.countplot(x = "category", data = df)
plt.xticks(rotation = 80);

In [ ]:
fig = px.histogram(df, x="category", title='Game Categories')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

In [ ]:
## total ratigs 

In [ ]:
df["total ratings"].describe()

In [ ]:
# HİSTOGRAMDA GÖSTERECEĞİZ

plt.figure(figsize=(20,16), dpi =400)
sns.displot(x='total ratings', data= df, bins=200, kde= True)

In [ ]:
plt.figure(figsize=(15,5))
sns.histplot(data=df, x ="total ratings", bins = 50);

In [ ]:
plt.figure(figsize=(17,4))
sns.boxplot(data=df, x ="total ratings");

In [ ]:
fig = px.histogram(df, x= 'total ratings', title='Total Ratings of the Games')
fig.show()

In [ ]:
fig = px.box(df, x= 'total ratings', hover_data = df[['title','category']])
fig.update_traces(quartilemethod="inclusive")
fig.show()

In [ ]:
## instal ile ilgili açıklayıcı grafik

İNSTALATİON ile ilgili görselleştirmeleri inceleyelim.

In [ ]:
df['installs_in_million'].describe()

In [ ]:
df.installs_in_million.value_counts()

In [ ]:
plt.figure(figsize = (16,6))
sns.countplot( x = df['installs_in_million']);

In [ ]:
fig = px.box(df, x= 'installs_in_million', hover_data = df[['title','installs_in_million']])
fig.update_traces(quartilemethod="inclusive")
fig.show()

In [ ]:
plt.figure(figsize=(15,5))
sns.histplot(data=df, x ="installs_in_million");

In [ ]:
fig = px.box(df, x= 'installs_in_million', hover_data = df[['title','category']])
fig.update_traces(quartilemethod="inclusive")
fig.show()

Paid an Games

In [ ]:
df['paid'].describe()

In [ ]:
df['paid'].value_counts()

In [ ]:
df['paid'].value_counts(normalize=True)

In [ ]:
values = [7, 1723]
index = ["Paid", "Free"]

In [ ]:
index, values

In [ ]:
plt.figure(figsize=(7,5))
y = values
mylabels = index
myexplode = [0, 0]
plt.pie(y, labels = mylabels, labeldistance=1.1, explode = myexplode, startangle=0, autopct='%1.1f%%')
plt.show()

In [ ]:
plt.figure(figsize=(7,5))
y = df["paid"].value_counts().values
mylabels = df["paid"].value_counts().index
myexplode = [0, 0]
plt.pie(y, labels = mylabels, labeldistance=1.1, explode = myexplode, startangle=0, autopct='%1.1f%%')
plt.show()

In [ ]:
paid_free= df['paid'].value_counts()
label =['Free','Paid']
fig = px.pie(paid_free, values=df['paid'].value_counts().values, names=label,
             title='Paid & Free Games')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
df["paid"].value_counts()

In [ ]:
plt.figure(figsize=(7,5))
y = df["paid"].value_counts().values
mylabels = df["paid"].value_counts().index
myexplode = [0, 0]

plt.pie(y, labels = mylabels, labeldistance=1.1, explode = myexplode, startangle=0, autopct='%1.1f%%')

plt.show()


In [ ]:
# creating pieplot using with plotly library
paid_free= df['paid'].value_counts()
label =['Free','Paid']
fig = px.pie(paid_free, values=df['paid'].value_counts().values, names=label,
             title='Paid & Free Games')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

## total ratings by category

category lere göre toplam ratingslerin 10 kategorinin her birinin toplam ratini var. 
kategorilerle toplam tarting gözküsün. group by la bunu çözebilirim.

In [ ]:
a=df.groupby ('category')['total ratings'].mean()
a

In [ ]:
type(a)

In [ ]:
a.index

In [ ]:
a.values

Kategorilere göre raytingleri inceleyelim. Oyun kategorilerine göre yükleme bilgileri incelenmiştir. Oyunların kategorilerine göre yüklenme bilgileri oyun şirketleri için yön çimek için faydalı olabilir. Yatırımlar ve tercihlerin belirlenmesine önemli olabilir. Veri setinde yaş sınırlandırması olsaydı çok daha net bir sınıflandırma ve analiz imkanı olabilirdi. Rakamlardan da anlaşılacağı üzere ACTİON oyunları en çok indirilen oyunlaların başında geliyor. Yatırım planı varsa bu alana yapılabilir. Eğitim alanındaki oyunlarda indirme oldukça düşük, Eğitim alanındaki oyunlarda gelir hedefi orta seviyede tutulmalıdır.

-------------------------
So I take was broadcast according to categories. Download information by game categories is examined. Loading information by categories of games can be useful for direction grass for game companies. It can be important to determine investments and preferences. If there were age restrictions in the data set, there would be a much clearer classification and analysis. As can be seen from the figures, action games are the most downloaded games. If you have an investment plan, it can be done in this area. In games in the training area, the download is quite low, and in games in the training area, the revenue target should be kept at a moderate level.

In [ ]:
b=df.groupby ('category')['installs_in_million'].mean().sort_values(ascending=False)
b

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data = df, x = a.index, y = a.values)
plt.xticks(rotation = 60);
plt.show()

In [ ]:
fig = px.bar(a, x= a.index, y=a.values, labels={'y':'Total Ratings'})
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

Kategoriye Göre En İyi Sıralanmış 5 Oyun

In [ ]:
BestFive = df[df['rank']<6][['rank','title','category', 'total ratings', 'installs_in_million', '5 star ratings']]
BestFive

In [ ]:
fig = px.scatter(BestFive, y= 'title', x='total ratings', 
                 hover_data = BestFive[['category','rank']], color='category', 
                 title = "Top 5 Games by Their Total Ratings")
fig.show()

In [ ]:
fig = px.scatter(BestFive, y= 'title', x='installs_in_million', 
                 hover_data = BestFive[['category','rank']], color='category', 
                 title = "Top 5 Games by Their Installations in Millions")
fig.show()

In [ ]:
fig = px.scatter(BestFive, y= 'title', x='5 star ratings', 
                 hover_data = BestFive[['category','rank']], color='category', 
                 title = "Top 5 Games by 5 Star Rankings")
fig.show()

In [ ]:
best50 = df.sort_values(by='installs_in_million', ascending=False).head(50)  

In [ ]:
best50

In [ ]:
fig = px.bar(best50, x= 'title', y='installs_in_million', hover_data = best50[['5 star ratings']], color='category')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(data = best50, x = "title", y = "installs_in_million", hue = "category", dodge = False)
plt.xticks(rotation = 90)
plt.show()

Teşekkür ederim.
Emeği geçen herkese özellikle KAAN beye teşekkür ederim. Başlangıç seviyesindeki bu projeyi incelerken keyif almanız umuyorum.

-----------
Thank you.
I would like to thank everyone who contributed, especially Mr. KAAN. I hope you enjoy this entry-level project.